In [0]:
#!import "Storage"

In [0]:
public interface BasicCashflows : IScope<CashflowIdentity, CashflowStorage>
{
    public IEnumerable<BasicCashflowDefinition> cd => GetStorage().GetBasicCashflowDefinitions(Identity); // TODO remove the IEnumerable here

    public FpaVariable[] rv => cd.Select((x,i) => new FpaVariable() { //  and here, create one rv
        Layer = i,
        Values = GenerateBasicCashflow(x),
        DataNode = Identity.DataNode,
        AmountType = Identity.AmountType,
        AccidentYear = Identity.AccidentYear,
        EstimateType = EstimateTypes.BE,
        Novelty = Novelties.C,
        AocType = AocTypes.CL
    }
    ).ToArray();
}

In [0]:
public interface BoundedCashflows : IScope<CashflowIdentity, CashflowStorage>
{
    public IEnumerable<BoundedCashflowDefinition> cd => GetStorage().GetBoundedCashflowDefinitions(Identity);

    public FpaVariable[] rv => cd.Select((x,i) => new FpaVariable() {
        Layer = i,
        Values = GenerateBoundedCashflow(x),
        DataNode = Identity.DataNode,
        AmountType = Identity.AmountType,
        AccidentYear = Identity.AccidentYear,
        EstimateType = EstimateTypes.BE,
        Novelty = Novelties.C,
        AocType = AocTypes.CL
    }
    ).ToArray();      
}

In [0]:
public interface ParametrisedCashflows : IScope<(CashflowIdentity Id, string originalAmountType), CashflowStorage>{

    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
                builder.ForScope<ParametrisedCashflows>(s => s.WithApplicability<ReferencedParametrisedCashflows>(x => x.Identity.originalAmountType != x.Identity.Id.AmountType));

    public double[] values => (GetScope<BasicCashflows>(Identity.Id).rv.Concat(
                               GetScope<BoundedCashflows>(Identity.Id).rv))
                               .Select(x => x.Values).AggregateDoubleArray(); // TODO zip parameters GetStorage().GetReferencedCashflowDefinitions(Identity)
}

public interface ReferencedParametrisedCashflows : ParametrisedCashflows
{

    double[] ParametrisedCashflows.values => GetScope<ReferencedCashflows>(Identity.Id).rv.Select(x => x.Values).AggregateDoubleArray();

}

public interface ReferencedCashflows : IScope<CashflowIdentity, CashflowStorage>
{
    public  IEnumerable<ReferencedCashflowDefinition> cd => GetStorage().GetReferencedCashflowDefinitions(Identity);

    public double[] values => cd.Select(x => GetScope<ParametrisedCashflows>((Identity with {AmountType = x.ReferenceAmountType}, x.AmountType)).values).AggregateDoubleArray();

    public FpaVariable[] rv => cd.Select((x, i) => new FpaVariable() {
        Layer = i,
        Values = values,
        DataNode = Identity.DataNode,
        AmountType = Identity.AmountType,
        AccidentYear = Identity.AccidentYear,
        EstimateType = EstimateTypes.BE,
        Novelty = Novelties.C,
        AocType = AocTypes.CL
    }
    ).ToArray(); 
}